In [1]:
import numpy as np
from matplotlib import pyplot as plt



# class definition

class ID3Model:

    edge_counter = 0
    tree = None
        
    def ID3(self, d, n, data):
        """ description: this function implements the ID3 algorithm based on the
            definition given in the assignment.
        input: 
            d := maximum depth of the tree, will be reduced on each level
            n := the maximum number of nodes. The static edge_counter variable
            shall not exceed the number of n - 1 edges.
            data := 2d numpy array {columns: features} X {rows: samples}
        output:
            tree := the binary decision tree for the dataset
        """
        
        # return a leaf in case of
        #   - uniform labels
        #   - maximal depth
        #   - maximum number of n nodes with n-1 edges
        #   - no more attributes 
        if (len(np.unique(data[:,0])) <= 1) or (d == 1) or (self.edge_counter >= n-1) or (data.shape[1] == 1):
            return self.majority_leaf(data)
        
        # else investigate node further
        else:
            
            # compute the information gain IG for each attribute
            gains = [self.compute_gain(data[:,0], data[:,x]) for x in range(1, data.shape[1])]    
            
            # get the attribute with highest IG
            split_column = np.argmax(gains)
            
            # separate row indices for 'y' and 'n' according
            # to the attribute with highest IG
            yea_idx = np.where(data[:,split_column] == 'y')[0]
            nea_idx = np.where(data[:,split_column] == 'n')[0]

            # return a leaf if samples are empty
            if len(nea_idx) == 0:
                return self.majority_leaf(data)
            if len(yea_idx) == 0:
                return self.majority_leaf(data)
            
            # split the data and remove the attribute with highest IG
            data_yea = np.delete(data[yea_idx,:], split_column, 1)
            data_nea = np.delete(data[nea_idx,:], split_column, 1)

            # label the decision node
            decision = str(split_column)

            # initialize sub-tree for the decision node
            tree = {decision: []}
            
            # increase the edge counter and reduce the max depth
            self.edge_counter +=2
            d -= 1
            
            # recursive calls on the split data
            yea = self.ID3(d, n, data_yea)
            nea = self.ID3(d, n, data_nea)
            
            # in case of equality return any leaf
            if yea == nea:
                tree = yea
            
            # else append 'yes' branch and 'no' branch
            else:
                tree[decision].append(yea)
                tree[decision].append(nea)
            return tree

    def compute_gain(self, S, i):
        """ description: compute the information gain H(S) - H(S|i)
        """
        H_S = self.entropy(S)

        # compute the conditional entropy H(S|i)
        avg_H = 0
        unique, counts = np.unique(i, return_counts=True)
        s = 1 / len(S)
        for j in range(len(unique)):
            idx = np.where(unique[j] == i)[0]
            avg_H += len(idx) * s * self.entropy(S[idx])

        return H_S - avg_H
    
    def entropy(self, X):

        # compute probability distribution of attribute X
        unique, counts = np.unique(X, return_counts=True)
        px = counts / len(X)

        # return the log base 2 entropy
        return -np.dot(px, np.log2(px))
    
    def majority_leaf(self, data):
        """ description: compute and return the most common label for a leaf
        """
        unique, counts = np.unique(data[:,0], return_counts=True)
        return unique[np.argmax(counts)]    
    
    def fit(self, d, n, data):
        """ description: train a tree based on the given data
        input: 
            d := maximum depth of the tree, will be reduced on each level
            n := the maximum number of nodes. The static edge_counter variable
            shall not exceed the number of n - 1 edges.
            data := 2d numpy array {columns: features} X {rows: samples}
        """
        self.edge_counter = 0
        self.tree = self.ID3(d, n, data)
        
        return self
        
    def evaluate(self, data):
        """ description: evaluate the model 
            for the given data
        """
        # tree not built
        if self.tree is None:
            print('no tree to evaluate')
            return np.nan
        
        fail_counter = 0
        
        # evaluate all samples from the data
        for x in range(data.shape[0]):
            sample = data[x]
            result = self.classify_sample(sample, self.tree)
            if result != sample[0]:
                fail_counter +=1

        return fail_counter / data.shape[0]
    
    def classify_sample(self, sample, tree):
        """ description: classify a sample with the given model
        """
        if (tree == 'democrat') or (tree == 'republican'):
            return tree
        
        decision = list(tree.keys())[0]
        attribute = int(decision)

        if sample[attribute] == 'y':
            result = tree[decision][0]
        if sample[attribute] == 'n':
            result = tree[decision][1]

        if (result == 'democrat') or (result == 'republican'):
            return result

        else:
            return self.classify_sample(sample, result)
        

        
        
        
# function definitions

def load_data(fname):

    data = np.genfromtxt(fname, delimiter=',', dtype=str)
        
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] == '?':
                
                # for each element set a random choice
                data[i][j] = np.random.choice(['y', 'n'])
                
    return data
        

def split_data(data, split):
    
    # create permuted indices for training and test sets
    perm = np.random.permutation(np.indices((len(data),))[0])
    
    # calculate the number of rows according to the given percentage
    nrows = int(split * len(data))
    training_set = data[perm[:nrows]]
    test_set = data[perm[nrows:]]
    return training_set, test_set


def learning_curve(d, n, training_set, test_set, increment=10):
    """ description: train and evaluate training and test data
        for different sizes of the test data and plot the results.
        input: 
            d := maximum depth of the tree, will be reduced on each level
            n := the maximum number of nodes. The static edge_counter variable
            shall not exceed the number of n - 1 edges.
            training_set := 2d numpy array {columns: features} X {rows: samples}
            test_set := 2d numpy array {columns: features} X {rows: samples}
            increment := the parameter 'r' as is defined in the assignment
    """
    # define min. data (==10) and max. data (==#rows of the training set)
    start = 10
    end = training_set.shape[0]
    m = range(start, end, increment)
    
    training_error = list(m)
    test_error = list(m)
    
    # istantiate the classifier
    t = ID3Model()
    
    for x in m:
        
        i = int((x - 10) / increment)
        
        # build tree and evaluate training and test errors
        training_error[i] = t.fit(d, n, training_set[0:x]).evaluate(training_set)
        test_error[i] = t.evaluate(test_set)
    
    # setup plot
    plt.figure()
    plt.xlabel('training set size')
    plt.ylabel('training / test error')
    plt.ylim([0, .6])
    plt.title('ID3(d:{},  n:{})'.format(d, n))

    # plot results
    plt.plot(m, training_error, '-.')
    plt.plot(m, test_error, 'r-')
    plt.legend(['training error', 'test error'])


    
    
    
# main program

# set seed before splitting the data for fixed 'random' values
np.random.seed(6666)

# load data
data = load_data('house-votes-84.data')

# split the data into training and test set
training_set, test_set = split_data(data, .7)

# initialize given parameters from the assignment
d = np.array([7, 4, 16, 7, 7])
n = np.array([40, 40, 40, 20, 130])

# call the learning_curve function for all 5 parameter constellations
for x in range(0,5):
    learning_curve(d[x], n[x], training_set, test_set, 1)
    #plt.savefig('../plots/fig2'+ str(x) +'.eps')

### References:
idea for building and evaluating a tree: https://github.com/SebastianMantey/Decision-Tree-from-Scratch/blob/master/notebooks/handling%20continuous%20and%20categorical%20variables.ipynb